# Calculating Delta Price and Delta Percent

This notebook shows how you can calculate change for historical assessments


### Initial Setup


In [1]:
import spgci as ci
from datetime import date


In [2]:
mdd = ci.MarketData()


### Getting a list of Symbols

This is a convenience to grab a list of a symbols to use with the `get_assessments_by_symbol_historical` method later on.


In [3]:
symbols = mdd.get_symbols(
    currency="USD", contract_type="Spot", commodity="Crude oil", page_size=10
)
symbols.head(10)


c:\Users\ANDREW_CHRISTIE\AppData\Local\pypoetry\Cache\virtualenvs\spgci-YFOrimOg-py3.8\lib\site-packages\spgci\api_client.py:106: UserWarning: 
Fetched page [1] of [175]. set `paginate=True` to fetch all pages.
  warnings.warn(


,symbol,description,assessment_frequency,bate_code,benchmark,holiday_schedule,commodity,commodity_grade,contract_type,currency,...,api_gravity,standard_lot_size,standard_lot_units,active,sulfur,timestamp,uom,delivery_region_port_id,density,delivery_region_basis_port_id
0,AJSVB00,Johan Sverdrup FOB North Sea vs North Sea Dtd ...,Daily (weekday),[c],Benchmark,CALLN00,Crude oil,Platts - Crude Oil - Johan Sverdrup,Spot,USD,...,28.0,600000.0,BBL,Active - Public,0.80,16:30 UK,BBL,NaN,NaN,NaN
1,AAGZU00,CPC Blend CIF Augusta,Daily (weekday),"[c, h, l]",Benchmark,CALLN00,Crude oil,Platts - Crude Oil - CPC Blend,Spot,USD,...,43.5,NaN,NaN,Active - Public,0.55,16:30 UK,BBL,MED,806.7,NaN
2,AALIN00,Urals Recombined (RCMB),Daily (weekday),"[c, h, l]",Benchmark,CALLN00,Crude oil,Platts - Crude Oil - Ural,Spot,USD,...,32.0,NaN,NaN,Active - Public,1.30,16:30 UK,BBL,MED,863.5,NaN
3,AWTID00,WTI Midland DAP Basis Rotterdam vs Fwd Dated B...,Daily (weekday),[c],Benchmark,CALLN00,Crude oil,Platts - Crude Oil - WTI Midland,Spot,USD,...,42.0,NaN,NaN,Active - Public,0.45,16:30 UK,BBL,NWE,1073.9,NaN
4,PCAFW00,Urals CIF Rdam,Daily (weekday),"[c, h, l]",Benchmark,CALLN00,Crude oil,Platts - Crude Oil - Ural,Spot,USD,...,32.0,NaN,NaN,Active - Public,1.30,16:30 UK,BBL,NL RTM,863.5,NL RTM
5,AWAFB00,WAF Index Differential $/bbl,Daily (weekday),[c],Benchmark,CALLN00,Crude oil,NaN,Spot,USD,...,NaN,NaN,NaN,Active - Public,NaN,16:30 UK,BBL,NaN,NaN,NaN
6,AWTIA00,WTI Midland DAP Basis Augusta,Daily (weekday),[c],Benchmark,CALLN00,Crude oil,Platts - Crude Oil - WTI Midland,Spot,USD,...,42.0,NaN,NaN,Active - Public,0.45,16:30 UK,BBL,MED,1073.9,NaN
7,AAHPL00,CPC Blend CIF Augusta vs BTC Dtd Strip,Daily (weekday),"[c, h, l]",Benchmark,CALLN00,Crude oil,Platts - Crude Oil - CPC Blend,Spot,USD,...,43.5,NaN,NaN,Active - Public,0.55,16:30 UK,BBL,MED,806.7,NaN
8,PCAAS00,Dated Brent,Daily (weekday),"[c, h, l]",Benchmark,CALCA00,Crude oil,Platts - Crude Oil - Brent,Spot,USD,...,38.0,NaN,NaN,Active - Public,0.45,16:30 UK,BBL,NaN,832.9,GB SUL
9,AJSVA00,Johan Sverdrup FOB North Sea,Daily (weekday),[c],Benchmark,CALLN00,Crude oil,Platts - Crude Oil - Johan Sverdrup,Spot,USD,...,28.0,600000.0,BBL,Active - Public,0.80,16:30 UK,BBL,NaN,NaN,NaN


### Getting Assessments and Calculating Delta

Using the symbols above get asessment history


In [4]:
df = mdd.get_assessments_by_symbol_historical(
    symbol=symbols["symbol"][:10], assess_date_gte=date(2023, 1, 1)
)
df


,bate,value,assessDate,isCorrected,modDate,symbol
0,c,76.090,2023-01-03,N,2023-01-03 18:43:32,AAGZU00
1,h,76.110,2023-01-03,N,2023-01-03 18:43:32,AAGZU00
2,l,76.070,2023-01-03,N,2023-01-03 18:43:32,AAGZU00
3,c,70.445,2023-01-04,N,2023-01-04 18:50:34,AAGZU00
4,h,70.460,2023-01-04,N,2023-01-04 18:50:34,AAGZU00
...,...,...,...,...,...,...
1455,h,58.670,2023-04-14,N,2023-04-14 17:54:25,PCAFW00
1456,l,58.640,2023-04-14,N,2023-04-14 17:54:25,PCAFW00
1457,c,57.780,2023-04-17,N,2023-04-17 17:19:44,PCAFW00
1458,h,57.800,2023-04-17,N,2023-04-17 17:19:44,PCAFW00


grouping by symbol and bate to calculate the deltaPrice and deltaPercent

_note: the NaN values are expected. since we are calculating this in pandas we do not know the assessment prior to the first record_


In [5]:
df = df.sort_values(
    "assessDate", ascending=True
)  # data should be sorted but just in case

grp = df.groupby(["symbol", "bate"])  # group by symbol AND bate

df["deltaPrice"] = grp["value"].diff()
df["deltaPercent"] = grp["value"].pct_change()
df["pDate"] = grp["assessDate"].shift(1)
df["pValue"] = grp["value"].shift(1)
df


,bate,value,assessDate,isCorrected,modDate,symbol,deltaPrice,deltaPercent,pDate,pValue
0,c,76.090,2023-01-03,N,2023-01-03 18:43:32,AAGZU00,NaN,NaN,NaT,NaN
1241,c,46.235,2023-01-03,N,2023-01-03 18:43:32,PCAFW00,NaN,NaN,NaT,NaN
1024,l,81.220,2023-01-03,N,2023-01-03 18:32:18,PCAAS00,NaN,NaN,NaT,NaN
1023,h,81.240,2023-01-03,N,2023-01-03 18:32:18,PCAAS00,NaN,NaN,NaT,NaN
1022,c,81.230,2023-01-03,N,2023-01-03 18:32:18,PCAAS00,NaN,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...
948,c,88.165,2023-04-17,N,2023-04-17 16:51:50,AWTIA00,-1.415,-0.015796,2023-04-14,89.580
1021,c,2.725,2023-04-17,N,2023-04-17 16:51:50,AWTID00,0.045,0.016791,2023-04-14,2.680
1238,c,86.555,2023-04-17,N,2023-04-17 17:19:27,PCAAS00,-1.330,-0.015133,2023-04-14,87.885
1240,l,86.550,2023-04-17,N,2023-04-17 17:19:27,PCAAS00,-1.330,-0.015134,2023-04-14,87.880


Showing the results


In [6]:
df.query("symbol == 'PCAAS00'").sort_values("assessDate", ascending=False).head(20)


,bate,value,assessDate,isCorrected,modDate,symbol,deltaPrice,deltaPercent,pDate,pValue
1240,l,86.550,2023-04-17,N,2023-04-17 17:19:27,PCAAS00,-1.330,-0.015134,2023-04-14,87.880
1238,c,86.555,2023-04-17,N,2023-04-17 17:19:27,PCAAS00,-1.330,-0.015133,2023-04-14,87.885
1239,h,86.560,2023-04-17,N,2023-04-17 17:19:27,PCAAS00,-1.330,-0.015133,2023-04-14,87.890
1235,c,87.885,2023-04-14,N,2023-04-14 17:54:03,PCAAS00,0.015,0.000171,2023-04-13,87.870
1236,h,87.890,2023-04-14,N,2023-04-14 17:54:03,PCAAS00,0.010,0.000114,2023-04-13,87.880
1237,l,87.880,2023-04-14,N,2023-04-14 17:54:03,PCAAS00,0.020,0.000228,2023-04-13,87.860
1234,l,87.860,2023-04-13,N,2023-04-13 17:52:42,PCAAS00,-0.340,-0.003855,2023-04-12,88.200
1233,h,87.880,2023-04-13,N,2023-04-13 17:52:42,PCAAS00,-0.330,-0.003741,2023-04-12,88.210
1232,c,87.870,2023-04-13,N,2023-04-13 17:52:42,PCAAS00,-0.335,-0.003798,2023-04-12,88.205
1230,h,88.210,2023-04-12,N,2023-04-12 17:54:04,PCAAS00,2.120,0.024625,2023-04-11,86.090
